## Analysis of Synthetic Data Generated from the Berka Dataset

In [1]:
import sqlalchemy
import psycopg2
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from genai_evaluation import multivariate_ecdf, ks_statistic
from nogan_synthesizer.preprocessing import wrap_category_columns, unwrap_category_columns

In [2]:
tables = [
    'account', 
    'card', 
    'client', 
    'disposition', 
    'district',
    'loan', 
    'orders', 
    'transaction'
]

In [3]:
def get_primary_keys_issues(engine, schema):
    try:
        query = text(f""" 
        select 'account' table_name, count(distinct account_id) as count_dist_ids, count(*) as count_rows
        from account
        having count(distinct account_id) <> count(*)
        union all 
        select 'card' table_name, count(distinct card_id) as count_dist_ids, count(*) as count_rows
        from card
        having count(distinct card_id) <> count(*)
        union all 
        select 'client' table_name, count(distinct client_id) as count_dist_ids, count(*) as count_rows
        from client
        having count(distinct client_id) <> count(*)
        union all 
        select 'disposition' table_name, count(distinct disposition_id) as count_dist_ids, count(*) as count_rows
        from disposition
        having count(distinct disposition_id) <> count(*)
        union all 
        select 'district' table_name, count(distinct district_id) as count_dist_ids, count(*) as count_rows
        from district
        having count(distinct district_id) <> count(*)
        union all 
        select 'loan' table_name, count(distinct loan_id) as count_dist_ids, count(*) as count_rows
        from loan
        having count(distinct loan_id) <> count(*)
        union all 
        select 'orders' table_name, count(distinct orders_id) as count_dist_ids, count(*) as count_rows
        from orders
        having count(distinct orders_id) <> count(*)
        union all 
        select 'transaction' table_name, count(distinct transaction_id) as count_dist_ids, count(*) as count_rows
        from transaction
        having count(distinct transaction_id) <> count(*)
        """)
        # print(query)
        with engine.connect() as conn:
            result = conn.execute(query)
            columns = result.keys()
            result_set = [{column: value for column, value in zip(columns, row)} for row in result]
            return result_set
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def get_foreign_keys_issues(engine, schema):
    try:
        query = text(f""" 
            select 'account_district' table_mappings, count(*) record_count 
            	from account where district_id not in (select district_id from district)
            having count(*) > 0
            union all
            select 'client_district' table_mappings, count(*) record_count 
            	from client where district_id not in (select district_id from district)
            having count(*) > 0	
            union all
            select 'disposition_account' table_mappings, count(*) record_count 
            	from disposition where account_id not in (select account_id from account)
            having count(*) > 0	
            union all
            select 'transaction_account' table_mappings, count(*) record_count 
            	from transaction where account_id not in (select account_id from account)
            having count(*) > 0	
            union all
            select 'loan_account' table_mappings, count(*) record_count 
            	from loan where account_id not in (select account_id from account)
            having count(*) > 0	
            union all
            select 'orders_account' table_mappings, count(*) record_count 
            	from orders where account_id not in (select account_id from account)
            having count(*) > 0	
            union all
            select 'disposition_client' table_mappings, count(*) record_count 
            	from disposition where client_id not in (select client_id from client)
            having count(*) > 0	
            union all
            select 'card_disposition' table_mappings, count(*) record_count 
            	from card where disposition_id not in (select disposition_id from disposition)
            having count(*) > 0        
        """)
        # print(query)
        with engine.connect() as conn:
            result = conn.execute(query)
            columns = result.keys()
            result_set = [{column: value for column, value in zip(columns, row)} for row in result]
            return result_set
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def get_constraints_count(engine, schema):
    try:
        query = text(f""" 
        select constraint_type, count(*) no_of_constraints from information_schema.table_constraints
        where constraint_schema  = '{schema}' and constraint_type in ('PRIMARY KEY', 'FOREIGN KEY')
        group by constraint_type
        """)
        # print(query)
        with engine.connect() as conn:
            result = conn.execute(query)
            columns = result.keys()
            result_set = [{column: value for column, value in zip(columns, row)} for row in result]
            return result_set
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [4]:
SOURCE_DBUSER = "genai_user"
SOURCE_DBPASSWD = "welcome1"
SOURCE_DBHOST = "217.76.56.228"
SOURCE_DBSCHEMA = "berka_original"
SOURCE_DBNAME = "genaidb"
SOURCE_DBCONN = f"postgresql+psycopg2://{SOURCE_DBUSER}:{SOURCE_DBPASSWD}@{SOURCE_DBHOST}/{SOURCE_DBNAME}"
SOURCE_CONNECT_ARGS = {'options': '-csearch_path={}'.format(SOURCE_DBSCHEMA)}

source_engine = create_engine(SOURCE_DBCONN, connect_args=SOURCE_CONNECT_ARGS)

source_data = {}
with source_engine.connect() as conn:
    for table in tables:
        source_data[table] = pd.read_sql_table(table_name = table, con =  conn, schema = SOURCE_DBSCHEMA)

In [5]:
SYNTH_DBUSER = "genai_user"
SYNTH_DBPASSWD = "welcome1"
SYNTH_DBHOST = "77.237.241.186"
SYNTH_DBNAME = "genai_resultdb"
SYNTH_DBCONN = f"postgresql+psycopg2://{SYNTH_DBUSER}:{SYNTH_DBPASSWD}@{SYNTH_DBHOST}/{SYNTH_DBNAME}"

#### Gretel Connection & Data

In [6]:
GRETEL_SYNTH_DBSCHEMA = "berka_gretel"
GRETEL_SYNTH_CONNECT_ARGS = {'options': '-csearch_path={}'.format(GRETEL_SYNTH_DBSCHEMA)}
gretel_engine = create_engine(SYNTH_DBCONN, connect_args=GRETEL_SYNTH_CONNECT_ARGS)

gretel_synth_data = {}
with gretel_engine.connect() as conn:
    for table in tables:
        gretel_synth_data[table] = pd.read_sql_table(table_name = table, con =  conn, schema = GRETEL_SYNTH_DBSCHEMA)

#### MostlyAI Connection & Data

In [7]:
MOSTLYAI_SYNTH_DBSCHEMA = "berka_mostlyai"
MOSTLYAI_SYNTH_CONNECT_ARGS = {'options': '-csearch_path={}'.format(MOSTLYAI_SYNTH_DBSCHEMA)}
mostlyai_engine = create_engine(SYNTH_DBCONN, connect_args=MOSTLYAI_SYNTH_CONNECT_ARGS)

mostlyai_synth_data = {}
with mostlyai_engine.connect() as conn:
    for table in tables:
        mostlyai_synth_data[table] = pd.read_sql_table(table_name = table, con =  conn, schema = MOSTLYAI_SYNTH_DBSCHEMA)

#### YDATA Connection & Data

In [8]:
YDATA_SYNTH_DBSCHEMA = "berka_ydata"
YDATA_SYNTH_CONNECT_ARGS = {'options': '-csearch_path={}'.format(YDATA_SYNTH_DBSCHEMA)}
ydata_engine = create_engine(SYNTH_DBCONN, connect_args=YDATA_SYNTH_CONNECT_ARGS)

ydata_synth_data = {}
with ydata_engine.connect() as conn:
    for table in tables:
        ydata_synth_data[table] = pd.read_sql_table(table_name = table, con =  conn, schema = YDATA_SYNTH_DBSCHEMA)

In [9]:
print("----- Source Data Details ------")
for table in tables:
    print(f"{table} Shape: {source_data[table].shape}")

print("")
print("----- Gretel Synth Data Details ------")
for table in tables:
    print(f"{table} Shape: {gretel_synth_data[table].shape}")

print("")
print("----- MostlyAI Synth Data Details ------")
for table in tables:
    print(f"{table} Shape: {mostlyai_synth_data[table].shape}")

print("")
print("----- YData Synth Data Details ------")
for table in tables:
    print(f"{table} Shape: {ydata_synth_data[table].shape}")

----- Source Data Details ------
account Shape: (4500, 4)
card Shape: (892, 4)
client Shape: (5369, 3)
disposition Shape: (5369, 4)
district Shape: (77, 16)
loan Shape: (682, 7)
orders Shape: (6471, 6)
transaction Shape: (49498, 9)

----- Gretel Synth Data Details ------
account Shape: (4500, 4)
card Shape: (892, 4)
client Shape: (5369, 3)
disposition Shape: (5369, 4)
district Shape: (77, 16)
loan Shape: (682, 7)
orders Shape: (6471, 6)
transaction Shape: (49498, 9)

----- MostlyAI Synth Data Details ------
account Shape: (3960, 4)
card Shape: (616, 4)
client Shape: (6299, 3)
disposition Shape: (4730, 4)
district Shape: (77, 16)
loan Shape: (816, 7)
orders Shape: (7823, 6)
transaction Shape: (43560, 9)

----- YData Synth Data Details ------
account Shape: (3908, 4)
card Shape: (892, 4)
client Shape: (5413, 3)
disposition Shape: (5369, 4)
district Shape: (77, 16)
loan Shape: (90, 7)
orders Shape: (5404, 6)
transaction Shape: (42986, 9)


In [10]:
constraints = get_constraints_count(source_engine, SOURCE_DBSCHEMA)
if constraints:
    # print(constraints)
    constraints_df = pd.DataFrame(constraints)
    print(constraints_df.head())
    total_constraints = constraints_df.no_of_constraints.sum()
else:
    total_constraints = 0

print(f"Total Constraints: {total_constraints}")


  constraint_type  no_of_constraints
0     FOREIGN KEY                  8
1     PRIMARY KEY                  8
Total Constraints: 16


### Gretel Preprocess Categorical Columns for table account

# Evaluate

#### Set Seed

In [11]:
cols = source_data[table].columns
[col for col in cols if not col.endswith('_id')]

['date', 'type', 'operation', 'amount', 'balance', 'k_symbol', 'bank']

In [12]:
pd.core.common.random_state(None)
seed = 1047
ks_seed = 1034

In [13]:
def evaluate_synth(synth_solution, original_data, synth_data, tables,
                   original_data_engine, synth_data_engine, total_constraints,
                   synth_dbschema, verbose = False
                  ):
    ## Data Integrity
    if verbose:
        print(f"---------{synth_solution} Evaluation-----------")
        print(f"{synth_solution} Constraint Issues Details")
    primary_keys_issues = get_primary_keys_issues(synth_data_engine, synth_dbschema)
    if verbose and primary_keys_issues:
        print(f"Primary Key Issues: {primary_keys_issues}")
    primary_keys_issue_count = len(primary_keys_issues)

    foreign_keys_issues = get_foreign_keys_issues(synth_data_engine, synth_dbschema)
    if verbose and primary_keys_issues:
        print(f"Foreign Key Issues: {foreign_keys_issues}")    
    foreign_keys_issue_count = len(foreign_keys_issues)
    data_integrity_score = 1 - ((primary_keys_issue_count + foreign_keys_issue_count)/total_constraints)
    if verbose:
        print(f" -Primary Keys Issues Count: {primary_keys_issue_count}")
        print(f" -Foreign Keys Issues Count: {foreign_keys_issue_count}")
        print(f" -Data Integrity Score: {data_integrity_score}")

    genai_eval_scores = []
    for table in tables:
        # print(table)
        orig_df = original_data[table]
        synth_df = synth_data[table]

        # Remove id columns
        non_id_cols = [col for col in orig_df.columns if not col.endswith('_id')]
        orig_df = orig_df[non_id_cols]
        synth_df = synth_df[non_id_cols]
        cat_columns = orig_df.select_dtypes(exclude=["number","bool_","datetime64[ns]"]).columns.tolist()
        # cat_columns = [col for col in cat_columns if not col.endswith('_id')]
        # print(cat_columns)
        date_columns = orig_df.select_dtypes(include=["datetime64[ns]"]).columns.tolist()
        if date_columns:
            orig_df = orig_df.drop(date_columns, axis = 1)
            synth_df = synth_df.drop(date_columns, axis = 1)
        
        # Encode Category Columns
        if cat_columns:
            wrapped_orig, idx_to_key_orig, key_to_idx_orig = wrap_category_columns(orig_df,cat_columns)
            wrapped_synth, idx_to_key_synth, key_to_idx_synth = wrap_category_columns(synth_df,cat_columns)
        else:
            wrapped_orig = orig_df
            wrapped_synth = synth_df
        # Calculate ECDF
        if verbose:
            print("----Calculating ECDF------")
        query_orig, ecdf_orig, ecdf_synth = \
            multivariate_ecdf(wrapped_orig, 
                              wrapped_synth, 
                              n_nodes = 3000, 
                              verbose = verbose,
                              random_seed=ks_seed) 

        # Calculate KS Stat
        ks_stat = ks_statistic(ecdf_orig, ecdf_synth)
        
        genai_eval_scores.append({"table":table, "eval_score": ks_stat})
    return {"data_integrity_score": data_integrity_score, "eval_scores": genai_eval_scores}

In [14]:
gretel_results = evaluate_synth(synth_solution = "gretel", original_data = source_data, 
                                synth_data = gretel_synth_data, tables = tables, original_data_engine = source_engine, 
                                synth_data_engine = gretel_engine, total_constraints = total_constraints,
                                synth_dbschema = GRETEL_SYNTH_DBSCHEMA, verbose=False)
               

In [15]:
print(f"Gretel Data Integrity Score: {gretel_results['data_integrity_score']}")
print(f"Gretel GenAI Eval Score: {pd.DataFrame(gretel_results['eval_scores']).eval_score.median():.3f}")


Gretel Data Integrity Score: 1.0
Gretel GenAI Eval Score: 0.270


In [16]:
mostlyai_results = evaluate_synth(synth_solution = "mostlyai", original_data = source_data, 
                                  synth_data = mostlyai_synth_data, tables = tables,
                                  original_data_engine = source_engine, synth_data_engine = mostlyai_engine, 
                                  total_constraints = total_constraints,
                                  synth_dbschema = MOSTLYAI_SYNTH_DBSCHEMA, verbose=False)
print(f"MostlyAI Data Integrity Score: {mostlyai_results['data_integrity_score']}")
print(f"MostlyAI GenAI Eval Score: {pd.DataFrame(mostlyai_results['eval_scores']).eval_score.mean():.3f}")

MostlyAI Data Integrity Score: 1.0
MostlyAI GenAI Eval Score: 0.314


In [17]:
ydata_results = evaluate_synth(synth_solution = "ydata", original_data = source_data, 
                               synth_data = ydata_synth_data, tables = tables,
                               original_data_engine = source_engine, synth_data_engine = ydata_engine, 
                               total_constraints = total_constraints,
                               synth_dbschema = YDATA_SYNTH_DBSCHEMA, verbose=False)
print(f"YData Data Integrity Score: {ydata_results['data_integrity_score']}")
print(f"YData GenAI Eval Score: {pd.DataFrame(ydata_results['eval_scores']).eval_score.mean():.3f}")

YData Data Integrity Score: 1.0
YData GenAI Eval Score: 0.160


In [18]:
gretel_synth_time = "40 mins"
mostlyai_synth_time = "2 mins"
ydata_synth_time = "7 mins"
results = [
    {"method": "Gretel", "synth_time": gretel_synth_time, 
     "data_integrity_score": gretel_results['data_integrity_score'], 
     "genai_eval_score": pd.DataFrame(gretel_results['eval_scores']).eval_score.mean().round(2)},
    {"method": "Mostly AI", "synth_time": mostlyai_synth_time, 
     "data_integrity_score": mostlyai_results['data_integrity_score'], 
     "genai_eval_score": pd.DataFrame(mostlyai_results['eval_scores']).eval_score.mean().round(2) },
    {"method": "YData AI", "synth_time": ydata_synth_time, 
     "data_integrity_score": ydata_results['data_integrity_score'], 
     "genai_eval_score": pd.DataFrame(ydata_results['eval_scores']).eval_score.mean().round(2) }  
]

pd.DataFrame(results)

,method,synth_time,data_integrity_score,genai_eval_score
0,Gretel,40 mins,1.0,0.28
1,Mostly AI,2 mins,1.0,0.31
2,YData AI,7 mins,1.0,0.16
